In [98]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from IPython.core.pylabtools import figsize
from IPython.display import IFrame
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn import neighbors                           
from sklearn import tree, linear_model,neighbors   
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_boston
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import*
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from mlxtend.frequent_patterns import apriori, association_rules

In [50]:
#Preparing for data, I have saved the prepared data. Please skip this chunk.
sales_with_category = pd.read_csv("sales_with_category.csv")
sales_category_list_cust_date_OH = sales_with_category[sales_with_category['StateProvince']=='OH'].groupby(['CustomerName','Sale Date','Base.Implement','EquipmentCategoryDesc']).size()
cust_purchase_history_date = sales_category_list_cust_date_OH.to_frame().unstack().unstack()
#cust_purchase_history_date.info()
cust_purchase_history_date1 = cust_purchase_history_date.dropna(axis = 1,how = 'all').sort_index(level = 2,axis = 1)
cust_purchase_history_date1.info()
cust_purchase_history_date1
cust_purchase_history_date1.to_csv('associate_rule_with_date&category_OH.csv')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 692 entries, ('A GEATCHES FARM LLC', '09/23/2019') to ('yoho don', '10/06/2020')
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   (0, AG Tractors, Base)                   124 non-null    float64
 1   (0, Combine Harvesting, Base)            2 non-null      float64
 2   (0, Compact 4WD Loaders, Base)           1 non-null      float64
 3   (0, Compact Excavators, Base)            14 non-null     float64
 4   (0, Compact Track Loaders, Base)         3 non-null      float64
 5   (0, Hay & Forage, Base)                  35 non-null     float64
 6   (0, ISG, Base)                           3 non-null      float64
 7   (0, Mowers, Base)                        300 non-null    float64
 8   (0, Nutrient Application, Base)          3 non-null      float64
 9   (0, PRECISION AG TECHNOLOGY, Base)       2 non-null      floa

0                     \
EquipmentCategoryDesc          AG Tractors Combine Harvesting   
Base.Implement                        Base               Base   
CustomerName        Sale Date                                   
A GEATCHES FARM LLC 09/23/2019         NaN                NaN   
ACKMAN MARCUS       04/07/2020         NaN                NaN   
ADAMS HAROLD        06/04/2020         NaN                NaN   
ADAMS LEAH          07/16/2021         NaN                NaN   
ADELMAN FRED        10/01/2019         NaN                NaN   
...                                    ...                ...   
rubeck michael      04/20/2020         NaN                NaN   
seabrook james      05/14/2020         NaN                NaN   
seabrook jim        05/14/2020         1.0                NaN   
sheffield john      04/28/2020         NaN                NaN   
yoho don            10/06/2020         NaN                NaN   

                                                                       \
EquipmentCategoryDesc          Compact 4WD Loaders Compact Excavators   
Base.Implement                                Base               Base   
CustomerName        Sale Date                                           
A GEATCHES FARM LLC 09/23/2019                 NaN                NaN   
ACKMAN MARCUS       04/07/2020                 NaN                NaN   
ADAMS HAROLD        06/04/2020                 NaN                NaN   
ADAMS LEAH          07/16/2021                 NaN                NaN   
ADELMAN FRED        10/01/2019                 NaN                NaN   
...                                            ...                ...   
rubeck michael      04/20/2020                 NaN                NaN   
seabrook james      05/14/2020                 NaN                NaN   
seabrook jim        05/14/2020                 NaN                NaN   
sheffield john      04/28/2020                 NaN                NaN   
yoho don            10/06/2020                 NaN                NaN   

                                                                               \
EquipmentCategoryDesc          Compact Track Loaders Hay & Forage  ISG Mowers   
Base.Implement                                  Base         Base Base   Base   
CustomerName        Sale Date                                                   
A GEATCHES FARM LLC 09/23/2019                   NaN          NaN  NaN    NaN   
ACKMAN MARCUS       04/07/2020                   NaN          NaN  NaN    1.0   
ADAMS HAROLD        06/04/2020                   NaN          NaN  NaN    1.0   
ADAMS LEAH          07/16/2021                   NaN          NaN  NaN    NaN   
ADELMAN FRED        10/01/2019                   NaN          NaN  NaN    NaN   
...                                              ...          ...  ...    ...   
rubeck michael      04/20/2020                   NaN          NaN  NaN    NaN   
seabrook james      05/14/2020                   NaN          NaN  NaN    1.0   
seabrook jim        05/14/2020                   NaN          NaN  NaN    NaN   
sheffield john      04/28/2020                   NaN          NaN  NaN    NaN   
yoho don            10/06/2020                   NaN          NaN  NaN    1.0   

                                                                             \
EquipmentCategoryDesc          Nutrient Application PRECISION AG TECHNOLOGY   
Base.Implement                                 Base                    Base   
CustomerName        Sale Date                                                 
A GEATCHES FARM LLC 09/23/2019                  NaN                     NaN   
ACKMAN MARCUS       04/07/2020                  NaN                     NaN   
ADAMS HAROLD        06/04/2020                  NaN                     NaN   
ADAMS LEAH          07/16/2021                  NaN                     NaN   
ADELMAN FRED        10/01/2019                  NaN                     NaN   
...                                       

In [99]:
#start here.
cust_purchase_history_date2 = pd.read_csv('associate_rule_with_date&category_OH.csv',index_col=0)
cust_purchase_history_date2
associate_rule_cust_date = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3 = cust_purchase_history_date2.iloc[1:,0:]
associate_rule_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 692 entries, 9/23/19 to 10/6/20
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   AG Tractors               124 non-null    object
 1   Combine Harvesting        2 non-null      object
 2   Compact 4WD Loaders       1 non-null      object
 3   Compact Excavators        14 non-null     object
 4   Compact Track Loaders     3 non-null      object
 5   Hay & Forage              35 non-null     object
 6   ISG                       3 non-null      object
 7   Mowers                    300 non-null    object
 8   Nutrient Application      3 non-null      object
 9   PRECISION AG TECHNOLOGY   2 non-null      object
 10  Planting & Seeding        9 non-null      object
 11  Portable Power Equipment  15 non-null     object
 12  Skid Steers               6 non-null      object
 13  Sprayers                  2 non-null      object
 14  Tillage              

In [100]:
for i in range(cust_purchase_history_date3.shape[0]):
    for j in range(cust_purchase_history_date3.shape[1]):
        if pd.isna(cust_purchase_history_date3.iloc[i][j]):
            associate_rule_cust_date.iloc[i][j] = 0
        else:
            associate_rule_cust_date.iloc[i][j] = 1
            

In [101]:
associate_rule_cust_date

,AG Tractors,Combine Harvesting,Compact 4WD Loaders,Compact Excavators,Compact Track Loaders,Hay & Forage,ISG,Mowers,Nutrient Application,PRECISION AG TECHNOLOGY,...,Utility Vehicles,Agriculture Attachments,Agriculture Implements,Compact Attachments,Cutters & Shredders,Landscape Equipment,Material Handling,Snow Equipment,Turf implements,Worksite Pro Implements
Sale Date,,,,,,,,,,,,,,,,,,,,,
9/23/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4/7/20,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6/4/20,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7/16/21,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10/1/19,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/20/20,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5/14/20,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5/14/20,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
new_df_cust_date = associate_rule_cust_date
new_df_cust_date['Base'] = 0
new_df_cust_date['Implement'] = 0
new_df_cust_date = new_df_cust_date.astype('float') 
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 692 entries, 9/23/19 to 10/6/20
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AG Tractors               692 non-null    float64
 1   Combine Harvesting        692 non-null    float64
 2   Compact 4WD Loaders       692 non-null    float64
 3   Compact Excavators        692 non-null    float64
 4   Compact Track Loaders     692 non-null    float64
 5   Hay & Forage              692 non-null    float64
 6   ISG                       692 non-null    float64
 7   Mowers                    692 non-null    float64
 8   Nutrient Application      692 non-null    float64
 9   PRECISION AG TECHNOLOGY   692 non-null    float64
 10  Planting & Seeding        692 non-null    float64
 11  Portable Power Equipment  692 non-null    float64
 12  Skid Steers               692 non-null    float64
 13  Sprayers                  692 non-null    float64
 14  Tilla

In [103]:
#new_df_cust_date.iloc[0:,17:]

In [104]:
m = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,:17]) == 0):
        m.append(0)
    else:
        m.append(1)
n = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,17:]) == 0):
        n.append(0)
    else:
        n.append(1)
    
new_df_cust_date['Base'] = m
new_df_cust_date['Implement'] = n
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 692 entries, 9/23/19 to 10/6/20
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AG Tractors               692 non-null    float64
 1   Combine Harvesting        692 non-null    float64
 2   Compact 4WD Loaders       692 non-null    float64
 3   Compact Excavators        692 non-null    float64
 4   Compact Track Loaders     692 non-null    float64
 5   Hay & Forage              692 non-null    float64
 6   ISG                       692 non-null    float64
 7   Mowers                    692 non-null    float64
 8   Nutrient Application      692 non-null    float64
 9   PRECISION AG TECHNOLOGY   692 non-null    float64
 10  Planting & Seeding        692 non-null    float64
 11  Portable Power Equipment  692 non-null    float64
 12  Skid Steers               692 non-null    float64
 13  Sprayers                  692 non-null    float64
 14  Tilla

In [105]:
new_df_cust_date.index = pd.to_datetime(new_df_cust_date.index)

In [106]:
OH_baseonly = new_df_cust_date.iloc[0:,17:27]
support_OH_baseonly = apriori(OH_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_OH = apriori(new_df_cust_date.iloc[0:,:26], min_support = 0.005, use_colnames = True, verbose = 1)
association_OH = association_rules(support_OH, metric = "confidence", min_threshold = 0.1)


Processing 30 combinations | Sampling itemset size 2
Processing 30 combinations | Sampling itemset size 3


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.033237,0.179191,0.017341,0.521739,2.911641,0.011385,1.716238
1,(Cutters & Shredders),(AG Tractors),0.027457,0.179191,0.007225,0.263158,1.468591,0.002305,1.113955
2,(Landscape Equipment),(AG Tractors),0.030347,0.179191,0.007225,0.238095,1.328725,0.001788,1.077312
3,(Material Handling),(AG Tractors),0.046243,0.179191,0.017341,0.375000,2.092742,0.009055,1.313295
4,(Turf implements),(AG Tractors),0.078035,0.179191,0.008671,0.111111,0.620072,-0.005313,0.923410
5,(Turf implements),(Mowers),0.078035,0.433526,0.013006,0.166667,0.384444,-0.020824,0.679769


In [107]:
support_OH_baseonly

,support,itemsets
0,0.033237,(Agriculture Implements)
1,0.027457,(Cutters & Shredders)
2,0.030347,(Landscape Equipment)
3,0.046243,(Material Handling)
4,0.078035,(Turf implements)
5,0.833815,(Base)
6,0.017341,"(Base, Agriculture Implements)"
7,0.017341,"(Base, Material Handling)"
8,0.023121,"(Turf implements, Base)"


In [108]:
association_OH

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.033237,0.179191,0.017341,0.521739,2.911641,0.011385,1.716238
1,(Cutters & Shredders),(AG Tractors),0.027457,0.179191,0.007225,0.263158,1.468591,0.002305,1.113955
2,(Landscape Equipment),(AG Tractors),0.030347,0.179191,0.007225,0.238095,1.328725,0.001788,1.077312
3,(Material Handling),(AG Tractors),0.046243,0.179191,0.017341,0.375000,2.092742,0.009055,1.313295
4,(Turf implements),(AG Tractors),0.078035,0.179191,0.008671,0.111111,0.620072,-0.005313,0.923410
5,(Turf implements),(Mowers),0.078035,0.433526,0.013006,0.166667,0.384444,-0.020824,0.679769


In [109]:
Jan1 = new_df_cust_date.loc['2019-01']
Jan2 = new_df_cust_date.loc['2020-01']
Jan3 = new_df_cust_date.loc['2021-01']
Jan = pd.concat([Jan1,Jan2,Jan3],axis=0)


Feb1 = new_df_cust_date.loc['2019-02']
Feb2 = new_df_cust_date.loc['2020-02']
Feb3 = new_df_cust_date.loc['2021-02']
Feb = pd.concat([Feb1,Feb2,Feb3],axis=0)

Mar1 = new_df_cust_date.loc['2019-03']
Mar2 = new_df_cust_date.loc['2020-03']
Mar3 = new_df_cust_date.loc['2021-03']
Mar = pd.concat([Mar1,Mar2,Mar3],axis=0)

Apr1 = new_df_cust_date.loc['2019-04']
Apr2 = new_df_cust_date.loc['2020-04']
Apr3 = new_df_cust_date.loc['2021-04']
Apr = pd.concat([Apr1,Apr2,Apr3],axis=0)


May1 = new_df_cust_date.loc['2019-05']
May2 = new_df_cust_date.loc['2020-05']
May3 = new_df_cust_date.loc['2021-05']
May = pd.concat([May1,May2,May3],axis=0)

Jun1 = new_df_cust_date.loc['2019-06']
Jun2 = new_df_cust_date.loc['2020-06']
Jun3 = new_df_cust_date.loc['2021-06']
Jun = pd.concat([Jun1,Jun2,Jun3],axis=0)

Jul1 = new_df_cust_date.loc['2019-07']
Jul2 = new_df_cust_date.loc['2020-07']
Jul3 = new_df_cust_date.loc['2021-07']
Jul = pd.concat([Jul1,Jul2,Jul3],axis=0)

Aug1 = new_df_cust_date.loc['2019-08']
Aug2 = new_df_cust_date.loc['2020-08']
Aug3 = new_df_cust_date.loc['2021-08']
Aug = pd.concat([Aug1,Aug2,Aug3],axis=0)

Sep1 = new_df_cust_date.loc['2019-09']
Sep2 = new_df_cust_date.loc['2020-09']
Sep3 = new_df_cust_date.loc['2021-09']
Sep = pd.concat([Sep1,Sep2,Sep3],axis=0)

Oct1 = new_df_cust_date.loc['2019-10']
Oct2 = new_df_cust_date.loc['2020-10']
Oct3 = new_df_cust_date.loc['2021-10']
Oct = pd.concat([Oct1,Oct2,Oct3],axis=0)

Nov1 = new_df_cust_date.loc['2019-11']
Nov2 = new_df_cust_date.loc['2020-11']
Nov3 = new_df_cust_date.loc['2021-11']
Nov = pd.concat([Nov1,Nov2,Nov3],axis=0)

Dec1 = new_df_cust_date.loc['2019-12']
Dec2 = new_df_cust_date.loc['2020-12']
Dec3 = new_df_cust_date.loc['2021-12']
Dec = pd.concat([Dec1,Dec2,Dec3],axis=0)

spring = pd.concat([Mar,Apr,May],axis=0)
summer = pd.concat([Jun,Jul,Aug],axis=0)
fall = pd.concat([Sep,Oct,Nov],axis=0)
winter = pd.concat([Dec,Jan,Feb],axis=0)

In [110]:
spring_baseonly = summer.iloc[0:,17:27]
support_spring_baseonly = apriori(spring_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_spring = apriori(spring.iloc[0:,:26], min_support = 0.005, use_colnames = True, verbose = 1)
association_spring = association_rules(support_spring, metric = "confidence", min_threshold = 0.1)

summer_baseonly = summer.iloc[0:,17:27]
support_summer_baseonly = apriori(summer_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_summer = apriori(summer.iloc[0:,:26], min_support = 0.005, use_colnames = True, verbose = 1)
association_summer = association_rules(support_summer, metric = "confidence", min_threshold = 0.1)

fall_baseonly = fall.iloc[0:,17:27]
support_fall_baseonly = apriori(fall_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_fall = apriori(fall.iloc[0:,:26], min_support = 0.005, use_colnames = True, verbose = 1)
association_fall = association_rules(support_fall, metric = "confidence", min_threshold = 0.1)

winter_baseonly = winter.iloc[0:,17:27]
support_winter_baseonly = apriori(winter_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_winter = apriori(winter.iloc[0:,:26], min_support = 0.005, use_colnames = True, verbose = 1)
association_winter = association_rules(support_winter, metric = "confidence", min_threshold = 0.1)

Processing 30 combinations | Sampling itemset size 2
Processing 30 combinations | Sampling itemset size 3
Processing 30 combinations | Sampling itemset size 2
Processing 9 combinations | Sampling itemset size 3
Processing 42 combinations | Sampling itemset size 2
Processing 36 combinations | Sampling itemset size 3
Processing 12 combinations | Sampling itemset size 3
Processing 24 combinations | Sampling itemset size 3


In [111]:
association_spring

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AG Tractors),(Agriculture Implements),0.170370,0.040741,0.022222,0.130435,3.201581,0.015281,1.103148
1,(Agriculture Implements),(AG Tractors),0.040741,0.170370,0.022222,0.545455,3.201581,0.015281,1.825185
2,(Landscape Equipment),(AG Tractors),0.029630,0.170370,0.007407,0.250000,1.467391,0.002359,1.106173
3,(Material Handling),(AG Tractors),0.037037,0.170370,0.014815,0.400000,2.347826,0.008505,1.382716
4,(Turf implements),(AG Tractors),0.085185,0.170370,0.011111,0.130435,0.765595,-0.003402,0.954074
5,(Compact Attachments),(Compact Excavators),0.018519,0.029630,0.007407,0.400000,13.500000,0.006859,1.617284
6,(Compact Excavators),(Compact Attachments),0.029630,0.018519,0.007407,0.250000,13.500000,0.006859,1.308642
7,(Turf implements),(Mowers),0.085185,0.488889,0.011111,0.130435,0.266798,-0.030535,0.587778


In [112]:
association_summer

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.020408,0.146939,0.008163,0.400000,2.722222,0.005165,1.421769
1,(Material Handling),(AG Tractors),0.032653,0.146939,0.008163,0.250000,1.701389,0.003365,1.137415
2,(Turf implements),(Mowers),0.053061,0.481633,0.012245,0.230769,0.479140,-0.013311,0.673878


In [113]:
association_fall

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.044643,0.223214,0.017857,0.40,1.792000,0.007892,1.294643
1,(Cutters & Shredders),(AG Tractors),0.017857,0.223214,0.008929,0.50,2.240000,0.004943,1.553571
2,(Landscape Equipment),(AG Tractors),0.035714,0.223214,0.017857,0.50,2.240000,0.009885,1.553571
3,(AG Tractors),(Material Handling),0.223214,0.089286,0.026786,0.12,1.344000,0.006856,1.034903
4,(Material Handling),(AG Tractors),0.089286,0.223214,0.026786,0.30,1.344000,0.006856,1.109694
5,(Turf implements),(AG Tractors),0.071429,0.223214,0.017857,0.25,1.120000,0.001913,1.035714
6,(Turf implements),(Mowers),0.071429,0.339286,0.017857,0.25,0.736842,-0.006378,0.880952
7,(Landscape Equipment),(Cutters & Shredders),0.035714,0.017857,0.008929,0.25,14.000000,0.008291,1.309524
8,(Cutters & Shredders),(Landscape Equipment),0.017857,0.035714,0.008929,0.50,14.000000,0.008291,1.928571
9,"(AG Tractors, Landscape Equipment)",(Cutters & Shredders),0.017857,0.017857,0.008929,0.50,28.000000,0.008610,1.964286


In [95]:
association_winter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.022989,0.275862,0.022989,1.000000,3.625000,0.016647,inf
1,(AG Tractors),(Cutters & Shredders),0.275862,0.057471,0.034483,0.125000,2.175000,0.018629,1.077176
2,(Cutters & Shredders),(AG Tractors),0.057471,0.275862,0.034483,0.600000,2.175000,0.018629,1.810345
3,(AG Tractors),(Material Handling),0.275862,0.080460,0.045977,0.166667,2.071429,0.023781,1.103448
4,(Material Handling),(AG Tractors),0.080460,0.275862,0.045977,0.571429,2.071429,0.023781,1.689655
5,(Agriculture Implements),(Material Handling),0.022989,0.080460,0.011494,0.500000,6.214286,0.009645,1.839080
6,(Material Handling),(Agriculture Implements),0.080460,0.022989,0.011494,0.142857,6.214286,0.009645,1.139847
7,(Cutters & Shredders),(Material Handling),0.057471,0.080460,0.011494,0.200000,2.485714,0.006870,1.149425
8,(Material Handling),(Cutters & Shredders),0.080460,0.057471,0.011494,0.142857,2.485714,0.006870,1.099617
9,"(AG Tractors, Agriculture Implements)",(Material Handling),0.022989,0.080460,0.011494,0.500000,6.214286,0.009645,1.839080
